In [1]:
from fh import FH 
import fh
from treeBuilder import TreeBuilder as TB
import PydalChanel as pydal
import copy
import pickle
import arpeggiator
from spatializer import Spatializer

lp = FH()


oneHitShift = fh.oneHitShift

read = pydal.read
end = pydal.end
tempo = pydal.tempo
rootScale = lp.rootScale
stripSymbols = lambda beats : map(lambda beat : list(beat[1])[0], beats)

ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)
ch3 = pydal.newChannel(3)
ch4 = pydal.newChannel(4)
ch5 = pydal.newChannel(5)

idv = lambda a : a
codec = lambda buf: fh.noteListToHitList(fh.flattenByBeat(fh.notesByBeat(fh.hitListToNoteList(buf))))

In [ ]:
s = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp)
s.spatialize = False
s.separateChannels = True
lp.addForwardingHandler(0, s)
s.loadChordsFromFile("songV4.chords")

a = arpeggiator.Arpeggiator(0, pydal.pydalInstance, read("0 1 2", 1, "arp"))
a.normalForwardingBehavior = False
lp.addForwardingHandler(0, a)

In [2]:
s = Spatializer(60, [[0,1,2,3], [4,5,6,7], [8,9,10,11]], lp)
s.spatialize = False
s.separateChannels = True
lp.addForwardingHandler(1, s)
s.loadChordsFromFile("songV4.chords")
def toggleSustain():
    s.sustaining = not s.sustaining
def toggleSeparateChannels():
    s.separateChannels = not s.separateChannels
lp.topRowFunctions[0] = toggleSustain
lp.topRowFunctions[1] = toggleSeparateChannels


In [ ]:
# ch1.play(read("[a:0_0.25*2, b:1_1.5*3] ", 1, "max"))
ch1.stop()

In [ ]:
rootScale(0, 57, "minor")

In [ ]:
def toggleSustain():
    s.sustaining = not s.sustaining
def toggleSeparateChannels():
    s.separateChannels = not s.separateChannels
lp.topRowFunctions[0] = toggleSustain
lp.topRowFunctions[1] = toggleSeparateChannels

In [ ]:
s.heldNotesOff(1)

In [ ]:
lp.startChannel(1, lambda buf: fh.spatialize(buf, lp.roots[0], [[0,1,2,3], [4,5,6,7], [8,9,10,11]]), lp.loops[0][0])

In [ ]:
lp.stopChannel(1)

In [2]:
a = read("{bd (lt*2, lt, lt), hc <~, sn hc> ~ hc:2}")
b = read("{(bd ~, sn:3 <~, cp>)}")
c = read("((bd:2*2, [bd:3 bd:3] bd), [sn:5 [bd sn:3]], [[bd sn:3] bd], <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>)")
c2 = read("(bd:2*2, [bd:3 bd:3] bd) [sn:5 [bd sn:3]] [[bd sn:3] bd] <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>", 4)
d = read("{~ ~, bin ~ bin ~}")
e = read("{[bin <bin:1, bin, bin*2> bin bot, hc <hc, ho>], bd sn:3}")

'''
bd 3
lt
sn 5
cp
hc:1 
ho
bot
bin 
bend:2
1 freebie 
'''

demo = read("bend:2 bin")

f = read("aa:4_4.75*2 aa:5_5.75*3", 2, 'sample')
f2 = read("hc*2 cp*3", 2)
d2 = read("{~ ~ , bd ~ sn:3 ~}", 4)
g = read("{~ ~ , ab:0_2 ~ ab:3_5 ~}", 4, 'sample')

In [36]:
ch1.play(read("[bd bd bd, hc (hc, ho, hc*2), bot <bot*2, bot> bot bot*2]", 2))

In [37]:
ch1.stop()

In [ ]:
s.loadChordsFromFile("songV4.chords")

In [ ]:
rootScale(1, lp.roots[1], "maj5")

In [ ]:
lp.startChannel(1, lambda buf: fh.randTranspose(buf, lp.roots[0], lp.scales[0]), lp.loops[0][1])

In [ ]:
lp.stopChannel(1)

In [ ]:
lp.startChannel(1, lambda buf: fh.warp(buf, 3, 1), lp.loops[1][3])